In [1]:
import json, sys, os, pandas as pd
from IPython import display

# Run this notebook to unpack a user_data directory from a contextual collection

### Required: 

In [2]:
input_dir        = "/data/chime/matthew/savannah_contextual/user_data/"
output_dir       = "savannah_contextual_csvs"
event_end_date   = pd.Timestamp("201610050000Z") # Users without any tweets before this date are ignored.

### Optional: 

In [3]:
# Just set the dates to False to not use them
begin_date = pd.Timestamp("201608010000Z")
end_date   = pd.Timestamp("201705010000Z")

begin_date = False
end_date   = False

### What columns get written to the CSV: 
Note, if they're not top-level tweet properties, will need to define them in `write_user` function

In [4]:
columns = [
    'created_at', 
    'text', 
    'coordinates', 
    'id_str', 
    'screen_name', 
    'user_location', 
    'user_time_zone', 
    'followers_count', 
    'friends_count', 
    'tweet_count', 
    'favorite_count', 
    'source',
    'user_id_str'
  ]

In [5]:
if not os.path.exists(output_dir):
    os.mkdir(output_dir) 

In [6]:
def parse_user(user_jsonl_file):
    ts = []
    for line in open(user_jsonl_file):
        ts.append(json.loads(line.strip()))
    df = pd.DataFrame(ts)
    df['timestamp'] = df.created_at.apply(lambda x: pd.Timestamp(x))
    return df

In [7]:
def check_dates(user_df, begin_date=False, end_date=False):
    if(user_df.timestamp.min() > event_end_date):
        users_without_tweets_in_range.append(user_df.user[0]['screen_name'])
        return None

    if begin_date:
        user_df = user_df[user_df.timestamp > begin_date]
        
    if end_date: 
        user_df = user_df[user_df.timestamp < end_date]
        
    return user_df

In [ ]:
def write_user(user_df):
    
    user_df['screen_name']    = user_df.user.apply(lambda x: x['screen_name'])
    user_df['user_location']  = user_df.user.apply(lambda x: x['location'])
    user_df['user_time_zone'] = user_df.user.apply(lambda x: x['time_zone'])
    user_df['user_id_str']    = user_df.user.apply(lambda x: x['id_str'])
    user_df['followers_count']= user_df.user.apply(lambda x: x['followers_count'])
    user_df['friends_count']  = user_df.user.apply(lambda x: x['friends_count'])
    user_df['tweet_count']    = user_df.user.apply(lambda x: x['statuses_count'])
    user_df['friends_count']  = user_df.user.apply(lambda x: x['friends_count'])
    
    user_df.sort_values(by='timestamp')[columns].to_csv(output_dir + "/" + u['screen_name'].values[0]+ ".csv", index=None)

# Full Runtime

In [ ]:
count = 0
users_without_tweets_in_range = []
with open(output_dir+'/users_without_tweets_in_range.txt','w') as failedUsers:
    for path, sub, files in sorted(os.walk(input_dir)): 
        for file in files:
    #         print(path, file)
            # Parse User
            u = parse_user(path+'/'+file)

            # Check Dates
            u = check_dates(u, begin_date=begin_date, end_date=end_date)

            # Write the User CSV
            if u is not None:
                write_user(u)
                count += 1
            else:
                failedUsers.write(users_without_tweets_in_range[-1]+"\n")
                failedUsers.flush()

            sys.stderr.write("\rWritten {0} User(s), Coult not capture event tweets for {1} user(s)      ".format(count, len(users_without_tweets_in_range)))
            

Written 175 User(s), Coult not capture event tweets for 45 user(s)      